<a href="https://colab.research.google.com/github/piyushac151/FakeNewsDetector.ipynb/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install scikit-learn nltk beautifulsoup4 requests joblib


In [43]:
from google.colab import files
uploaded = files.upload()


Saving True.csv to True (3).csv


In [44]:
from google.colab import files
uploaded = files.upload() f

Saving Fake.csv to Fake (3).csv


In [46]:
import pandas as pd

fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

fake_df["label"] = 0
true_df["label"] = 1


In [47]:
df = pd.concat([fake_df, true_df], axis=0)

df = df.sample(frac=1).reset_index(drop=True)

df.head()


,title,text,subject,date,label
0,Iraq top shi'ite cleric Sistani asks governmen...,BAGHDAD (Reuters) - Iraq s top Shi ite cleric ...,worldnews,"October 20, 2017",1
1,BLACK RESIDENTS NOT HAPPY After Street Artist ...,The Obama s have convinced Black communities a...,left-news,"Sep 9, 2016",0
2,BREAKING: TSA AGENT SEXUALLY MOLESTS STUDENT A...,This is so scary for anyone out there but espe...,Government News,"Aug 28, 2015",0
3,Factbox - Republicans cut side deals to push t...,WASHINGTON (Reuters) - U.S. Senate Republicans...,politicsNews,"December 3, 2017",1
4,Germany's Schaeuble urges parties to compromis...,BERLIN (Reuters) - German parties must be more...,worldnews,"November 21, 2017",1


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


['vectorizer.pkl']

In [49]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import re

model = joblib.load("model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def predict_news(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    prediction = model.predict(vec)[0]
    prob = model.decision_function(vec)[0]
    confidence = 1 / (1 + np.exp(-abs(prob)))
    return prediction, confidence

st.title("📰 Fake News Detector")

user_input = st.text_area("Paste a news article or headline:")

if st.button("Check"):
    if user_input:
        label, confidence = predict_news(user_input)
        st.markdown(f"### 🔎 Prediction: {'Real' if label else 'Fake'}")
        st.markdown(f"**Confidence**: {confidence * 100:.2f}%")


Overwriting app.py


In [50]:
%%writefile requirements.txt
streamlit
scikit-learn
joblib
numpy


Writing requirements.txt


In [51]:
from google.colab import files

files.download('model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:

files.download('vectorizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:

files.download('app.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:

files.download('requirements.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>